# Data Preprocessing

## Load Library

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import pandas as pd
import time

In [3]:
# Read the datasets
df_house = pd.read_csv('../../ADS ASS2/Data/combined_house.csv')
df_apartment = pd.read_csv('../../ADS ASS2/Data/combined_apartment.csv')
df_townhouse = pd.read_csv('../../ADS ASS2/Data/combined_townhouse.csv')

In [4]:
df_house

,Price,Address,Beds,Baths,Parking
0,$270 per week,"50 Elmhurst Road, Bayswater North VIC 3153",1 Bed,1 Bath,− Parking
1,280,"63 Pintail Drive, Melton South VIC 3338",1 Bed,1 Bath,− Parking
2,300,"8 Jacobena Road, Melton South VIC 3338",1 Bed,1 Bath,− Parking
3,$400.00,"126 Portarlington Road, Newcomb VIC 3219",3 Beds,1 Bath,1 Parking
4,$395.00,"123 Mossfiel Drive, Hoppers Crossing VIC 3029",3 Beds,2 Baths,2 Parking
...,...,...,...,...,...
6948,from $200 per night,"72 McLachlan Street, Apollo Bay VIC 3233",4 Beds,1 Bath,2 Parking
6949,from $300 per night,"4 Ocean Park Drive, Marengo VIC 3233",5 Beds,2 Baths,2 Parking
6950,"$3,850","7 Karlson Street, Skenes Creek VIC 3233",3 Beds,2 Baths,4 Parking
6951,from $180 per night off peak,"45 Murray Square, Apollo Bay VIC 3233",3 Beds,1 Bath,2 Parking


## Merge Three Dataframes

In [5]:
# Add the new column to distinguish property types
df_house['Type'] = 'House'
df_apartment['Type'] = 'Apartment'
df_townhouse['Type'] = 'Townhouse'

# Merge the three dataframes
df_combined = pd.concat([df_house, df_apartment, df_townhouse], ignore_index=True)

df_combined.shape

(14029, 6)

## Data Cleaning

In [6]:
# Remove the rows for no data found
df_combined = df_combined[
    (df_combined['Price'] != 'No price found') &
    (df_combined['Address'] != 'No address found') &
    (df_combined['Beds'] != 'No beds info') &
    (df_combined['Baths'] != 'No baths info')
]

df_combined.shape

(14007, 6)

In [7]:
# Remove the car parks or storage rooms
df_combined = df_combined[~df_combined['Address'].str.contains('Carpark|Car Space|Storage Ca', case=False, na=False)]
df_combined.shape

(13978, 6)

In [8]:
# Extract the integer parts of beds, baths and parking
df_combined['Beds'] = df_combined['Beds'].str.extract('(\d+)').fillna(0).astype(int)
df_combined['Baths'] = df_combined['Baths'].str.extract('(\d+)').fillna(0).astype(int)
df_combined['Parking'] = df_combined['Parking'].str.extract('(\d+)').fillna(0).astype(int)

df_combined

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/var/folders/6h/l3lkzl091tq1btbgkvc4_pkr0000gn/T/ipykernel_15889/3452778338.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_combined['Beds'] = df_combined['Beds'].str.extract('(\d+)').fillna(0).astype(int)
/var/folders/6h/l3lkzl091tq1btbgkvc4_pkr0000gn/T/ipykernel_15889/3452778338.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_combined['Baths'] = df_combined['Baths'].str.extract('(\d+)').fillna(0).astype(int)
/var/folders/6h/l3lkzl091tq1btbgkvc4_pkr0000gn/T/ipykernel_15889/3452778338.py:4: SyntaxWarning: invalid escape sequence '\d'
  df_combined['Parking'] = df_combined['Parking'].str.extract('(\d+)').fillna(0).astype(int)


,Price,Address,Beds,Baths,Parking,Type
0,$270 per week,"50 Elmhurst Road, Bayswater North VIC 3153",1,1,0,House
1,280,"63 Pintail Drive, Melton South VIC 3338",1,1,0,House
2,300,"8 Jacobena Road, Melton South VIC 3338",1,1,0,House
3,$400.00,"126 Portarlington Road, Newcomb VIC 3219",3,1,1,House
4,$395.00,"123 Mossfiel Drive, Hoppers Crossing VIC 3029",3,2,2,House
...,...,...,...,...,...,...
14024,$770,"16 Amara Circuit, Oakleigh South VIC 3167",3,2,2,Townhouse
14025,$800,"78 Middleborough Road, Burwood East VIC 3151",3,2,1,Townhouse
14026,$900,"31 Botanic Drive, Glen Waverley VIC 3150",4,3,2,Townhouse
14027,$720,"32/29 Browns Road, Clayton VIC 3168",3,2,1,Townhouse


In [9]:
df_combined.shape

(13978, 6)

In [10]:
# Remove the price with no specific numbers
df_combined = df_combined[df_combined['Price'].str.contains(r'\d')]

df_combined.shape

(13698, 6)

In [11]:
def extract_weekly_price(price):
    # Change the price string to lower case
    price = price.lower().strip()
    
    # Replace the specific symbols
    price = price.replace('$', '').replace(',', '')
    
    # Delete the price with from beginning
    if 'from' in price:
        price = price.replace('from', '').strip()
    
    # Extract the part with weekly price first
    weekly_match = re.search(r'\((\d+)\s*pw\)', price)
    if weekly_match:
        return int(weekly_match.group(1))
    
    # Find the weekly price as many below situactions
    if 'pw' in price or 'per week' in price or 'week' in price or 'weekly' in price or 'p.w.' in price:
        parts = price.split()
        for part in parts:
            if 'pw' in part:
                try:
                    return int(float(part.replace('pw', '').strip()))
                except ValueError:
                    continue
            elif '.' in part:
                try:
                    return int(float(part))
                except ValueError:
                    continue
            elif part.isdigit():
                return int(part)

    # Find the monthly price and change them to weekly
    if 'month' in price or 'pm' in price:
        parts = price.split()
        for part in parts:
            if part.isdigit():
                monthly_value = int(part)
                weekly_value = round(monthly_value / 4.34)
                return weekly_value

    # Find the nightly price and change them to weekly
    nightly_match = re.search(r'(\d+)\s*(night)', price)
    if nightly_match:
        nightly_value = int(nightly_match.group(1))
        return nightly_value * 7

    # If no explicit unit, return the first value
    default_numbers = re.findall(r'\d+', price)
    if default_numbers:
        return int(default_numbers[0])
    else:
        return None

# Add the new column
df_combined['Weekly Price'] = df_combined['Price'].apply(extract_weekly_price)

df_combined


/var/folders/6h/l3lkzl091tq1btbgkvc4_pkr0000gn/T/ipykernel_15889/2257404655.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['Weekly Price'] = df_combined['Price'].apply(extract_weekly_price)


,Price,Address,Beds,Baths,Parking,Type,Weekly Price
0,$270 per week,"50 Elmhurst Road, Bayswater North VIC 3153",1,1,0,House,270
1,280,"63 Pintail Drive, Melton South VIC 3338",1,1,0,House,280
2,300,"8 Jacobena Road, Melton South VIC 3338",1,1,0,House,300
3,$400.00,"126 Portarlington Road, Newcomb VIC 3219",3,1,1,House,400
4,$395.00,"123 Mossfiel Drive, Hoppers Crossing VIC 3029",3,2,2,House,395
...,...,...,...,...,...,...,...
14024,$770,"16 Amara Circuit, Oakleigh South VIC 3167",3,2,2,Townhouse,770
14025,$800,"78 Middleborough Road, Burwood East VIC 3151",3,2,1,Townhouse,800
14026,$900,"31 Botanic Drive, Glen Waverley VIC 3150",4,3,2,Townhouse,900
14027,$720,"32/29 Browns Road, Clayton VIC 3168",3,2,1,Townhouse,720


In [12]:
# Filter the weekly price between 100 to 2000
df_combined = df_combined[(df_combined['Weekly Price'] >= 100) & (df_combined['Weekly Price'] <= 2000)]

df_combined.shape

(13466, 7)

In [13]:
df_combined.to_csv('../../ADS ASS2/Data/combined_domain.csv', index=False)

## Add Longitude and Latitude on Domain Datasets

In [ ]:
# Protect our company API Keys
API_KEY = 'YourKeys'

# Initialise an empty list of Latitude and Longitude
longitudes = []
latitudes = []

# Iterate through each page's pagination links
def get_lat_long(address):
    try:
        url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY}'
        response = requests.get(url)
        data = response.json()

        if len(data['results']) > 0:
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            return None, None
    except Exception as e:
        print(f"Error occurred for address {address}: {e}")
        return None, None

# Use Geocoding API to get the latitude and longitude
for address in df_combined['Address']:
    latitude, longitude = get_lat_long(address)
    latitudes.append(latitude)
    longitudes.append(longitude)
    time.sleep(0.1)

# Add these two columns in df_combined
df_combined['Latitude'] = latitudes
df_combined['Longitude'] = longitudes

print(df_combined.head())

In [ ]:
df_domain = df_combined.dropna(subset=['Latitude', 'Longitude'])

# Save the dataset to csv document
df_domain.to_csv('../../ADS ASS2/Data/updated_domain_with_lat_long.csv', index=False)

## Add Universities to School Dataset

In [3]:
df_school_locations = pd.read_csv('../../ADS ASS2/Data/School_Locations.csv')
df_school_locations

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1.0,1.0,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971.0,03 5183 2412,681.0,Wellington (S),146.66660,-38.61771
1,Government,1.0,3.0,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277.0,03 5565 1382,673.0,Warrnambool (C),142.59039,-38.38628
2,Government,1.0,4.0,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467.0,03 5465 3176,599.0,Pyrenees (S),143.47565,-37.08450
3,Government,1.0,8.0,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664.0,03 5796 2264,643.0,Strathbogie (S),145.23472,-36.90137
4,Government,1.0,12.0,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113.0,03 9844 3537,421.0,Manningham (C),145.21398,-37.74268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2303,NaN,NaN,NaN,University of Melbourne (Southbank),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.97018,-37.82464
2304,NaN,NaN,NaN,Monash University (Clayton),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.13514,-37.91406
2305,NaN,NaN,NaN,Monash University (Caulfield),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.04550,-37.87717
2306,NaN,NaN,NaN,RMIT (City),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.96393,-37.80799


In [ ]:
# Longtitude and Latitude of Universities in Victoria State
university_rows = [
    {'School_Name': 'University of Melbourne (Parkville)', 'X': '144.96099', 'Y': '-37.79830'},
    {'School_Name': 'University of Melbourne (Southbank)', 'X': '144.97018', 'Y': '-37.82464'},
    {'School_Name': 'Monash University (Clayton)', 'X': '145.13514', 'Y': '-37.91406'},
    {'School_Name': 'Monash University (Caulfield)', 'X': '145.04550', 'Y': '-37.87717'},
    {'School_Name': 'RMIT (City)', 'X': '144.96393', 'Y': '-37.80799'},
    {'School_Name': 'Deakin University', 'X': '144.95024', 'Y': '-37.82030'},
]

university_df = pd.DataFrame(university_rows)

df_new_university = pd.concat([df_school_locations, university_df], ignore_index=True)

# Reserve to the csv document
df_new_university.to_csv('../../ADS ASS2/Data/School_Locations.csv', index=False)